In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from folium.plugins import HeatMap

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
# Load districts of zurich and coordinates
districts = pd.read_csv('zh_quartiere.csv')
districts.head()

,Stadtquartier,QuarCD,BFSCode,PLZ,PLZ2,Kreis,Latitude,Longitude
0,Rathaus,NaN,261011,8001,NaN,1,47.376389,8.548056
1,Hochschulen,NaN,261012,8001,NaN,1,47.376389,8.548056
2,Lindenhof,NaN,261013,8001,NaN,1,47.364645,8.566284
3,City,NaN,261014,8001,NaN,1,47.364645,8.566284
4,Wollishofen,NaN,261021,8038,NaN,2,47.343944,8.530114


In [11]:
## Load data with nationalities per Zurich district
zhdf1 = pd.read_csv('https://data.stadt-zuerich.ch/dataset/eeb68173-d3e5-43f5-80cf-318b5a4045bf/resource/e0efc72f-29cd-4176-aa06-fc070d26278e/download/bev336od3361.csv')

## Limit to caribbean/south american nationalities
zhdf1 = zhdf1[zhdf1.RegionSort>31]
zhdf1 = zhdf1[zhdf1.RegionSort<40]
zhdf1 = zhdf1[zhdf1.StichtagDatJahr == 2018]

## Build summary for latin american population per district
zhdf1 = zhdf1.groupby('QuarLang').agg({"AnzBestWir": "sum"})
zhdf1.head(100)

,AnzBestWir
QuarLang,
Affoltern,314
Albisrieden,178
Alt-Wiedikon,172
Altstetten,413
City,9
Enge,78
Escher Wyss,69
Fluntern,32
Friesenberg,74


In [13]:
## Join dataframes to match coordinates and regions - this will give us the number of latin americans per district together with district coordinates
latins = pd.merge(zhdf1, districts, left_on='QuarLang', right_on='Stadtquartier')
latins.head()

,AnzBestWir,Stadtquartier,QuarCD,BFSCode,PLZ,PLZ2,Kreis,Latitude,Longitude
0,314,Affoltern,NaN,261111,8046,NaN,11,47.419094,8.506474
1,178,Albisrieden,NaN,261091,8047,NaN,9,47.375391,8.485338
2,172,Alt-Wiedikon,NaN,261031,8003,NaN,3,47.368441,8.517243
3,413,Altstetten,NaN,261092,8048,NaN,9,47.387530,8.485842
4,9,City,NaN,261014,8001,NaN,1,47.364645,8.566284


In [14]:
# Use geopy library to get the latitude and longitude values for Zurich

address = 'Zurich'

geolocator = Nominatim(user_agent="zh_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Zurich are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Zurich are 47.3723941, 8.5423328.


In [15]:
# create map of Zurich using latitude and longitude values
map_zurich = folium.Map(location=[latitude, longitude], zoom_start=12)
    
# add markers to map, showing city districts with number of latin americans per each district
for lat, lng, quartier, qnt in zip(latins['Latitude'], latins['Longitude'], latins['Stadtquartier'], latins['AnzBestWir']):
    label = '{}:{}'.format(quartier, qnt)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich)  
    
map_zurich

In [16]:
## create heatmap for latin-american population in Zurich - this will give us a density map and tell us in which areas of Zurich latin americans are most present
max_amount = float(latins['AnzBestWir'].max())

hm_wide = HeatMap( list(zip(latins.Latitude.values, latins.Longitude.values, latins.AnzBestWir.values)),
                   min_opacity=0.2,
                   max_val=max_amount,
                   radius=35, blur=15, 
                   max_zoom=2, 
                 )

map_zurich.add_child(hm_wide)

In [17]:
# Load data from zuerich.com for existing bars and lounges to see where bar businesses are located which target latin-american/american population

SSL_VERIFY = True
# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.zuerich.com nicht klappt (z.B. wegen Proxy)
# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)
SSL_VERIFY = False
if not SSL_VERIFY:
    import urllib3
    urllib3.disable_warnings()
    
def get_de(field):
    try:
        return field['de']
    except (KeyError, TypeError):
        try:
            return field['en']
        except (KeyError, TypeError):
            return field
        
## read district information for Zurich
headers = {'Accept': 'application/json'}
r = requests.get('https://data.stadt-zuerich.ch/dataset/statistisches_quartier/resource/c837926e-035d-48b9-8656-03f1b13c323b/download/statistische_quartiere.json', headers=headers, verify=SSL_VERIFY)

district_geo = r.json()
district_geo


{'name': 'StatistischesQuartier',
 'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'MultiPolygon',
    'coordinates': [[[[8.47023638671907, 47.3739506414208],
       [8.47033673166121, 47.3739588752243],
       [8.47053472534719, 47.3741281383317],
       [8.47065295785768, 47.3742292747057],
       [8.47072580155878, 47.3742915891487],
       [8.47110650589651, 47.3745777282919],
       [8.47136078217231, 47.3747558957459],
       [8.47165560292566, 47.3749625741451],
       [8.47179106624585, 47.3750575196082],
       [8.47188300088914, 47.3751201932907],
       [8.47187681848969, 47.3751291536723],
       [8.47180175260936, 47.3752385750228],
       [8.47214842904971, 47.3754279676684],
       [8.47257851266964, 47.3756231701708],
       [8.47284880074573, 47.3757449721048],
       [8.47338651894776, 47.3759441642244],
       [8.47333552303066, 47.3759645899739],
       [8.47335127818153, 47.3759774017055],
       [8.47336699216698, 47.37598877

In [18]:
## read JSON data for information of existing bars and lounges in Zurich
headers = {'Accept': 'application/json'}
r = requests.get('https://www.zuerich.com/en/data/gastronomy/nightlife/bars+%7C%7C+lounges', headers=headers, verify=SSL_VERIFY)

bardata = r.json()
bardata

[{'id': '527248',
  '@context': 'https://schema.org/',
  '@type': 'LocalBusiness',
  'copyright': {'en': 'Zurich Tourism www.zuerich.com',
   'de': 'Zürich Tourismus www.zuerich.com',
   'it': 'Zürich Tourismo www.zuerich.com',
   'fr': 'Zürich Tourisme www.zuerich.com'},
  'cc': 'BY-SA',
  'category': ['Nightlife',
   'Atmosphere',
   'Cool and Trendy',
   'Bars & Lounges',
   'Neighborhood Bar'],
  'name': {'de': 'Total Bar',
   'en': 'Total Bar',
   'fr': 'Total Bar',
   'it': 'Total Bar'},
  'disambiguatingDescription': {'de': 'Wie ein gallisches Dorf trotzt diese Bar seit 20 Jahren allem Trubel um sie herum. Zum Glück für Freunde der entspannten Bar-Kultur.',
   'en': 'Like the Gallic village of “Asterix and Obelix” fame, for 20 years this bar has withstood all the many changes going on around it. Which is good news for fans of the relaxed bar culture.',
   'fr': 'Tel un village gaulois, ce bar résiste depuis 20 ans au tumulte. Une chance pour les amateurs appréciant le côté décon

In [41]:
# Transform the bar data into a pandas dataframe

# define the dataframe columns
column_names = ['Name', 'Postal Code', 'Latitude', 'Longitude'] 

# instantiate the dataframe
barinfo = pd.DataFrame(columns=column_names)

# Load data into dataframe
for data in bardata:
    bar_name = data['name']['en']       
    bar_description = data['description']['en']
    bar_postalcode = data['address']['postalCode']
    bar_lat = float(data['geo']['latitude'])
    bar_lon = float(data['geo']['longitude'])
    
    if ("LATIN" in bar_description.upper()) or ("AMERICAN" in bar_description.upper()):    
        barinfo = barinfo.append({'Name': bar_name,
                                  'Postal Code': bar_postalcode,
                                  'Latitude': bar_lat,
                                  'Longitude': bar_lon}, ignore_index=True)
    
barinfo.head()

,Name,Postal Code,Latitude,Longitude
0,Escherwyss Club,8005,47.389899,8.521228
1,Jade Club,8001,47.371059,8.535909
2,PurPur,8008,47.364734,8.548579
3,Fat Tony,8004,47.380067,8.527701
4,Le Raymond Bar,8001,47.368860,8.537515


In [43]:
# add markers to map, showing existing bars on the map as yellow dots
for lat, lng, barname in zip(barinfo['Latitude'], barinfo['Longitude'], barinfo['Name']):
    label = folium.Popup(barname, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_zurich)  
    
map_zurich